In [ ]:
%%capture
%run 3-tfidf-gensim.ipynb

In [ ]:
from gensim.models import LdaModel, CoherenceModel
# %pip install pyldavis==3.3.1
import pyLDAvis, pyLDAvis.gensim

The LDA model uses Dirichlet distributions. The parameters of Dirichlet distributions can be either symmetrical, in which case all the values of the variable will have equal weight, or asymmetrical, where some values have higher weight than others. Moreover, the parameters can be equal to, smaller or greater than 1. For parameters equal to 1, the distribution will assign uniform probability. For values greater than 1, more probability will be assigned to the center, while for values smaller than 1 more probability will be assigned to the corners.

In the LDA model, alpha refers to the Dirichlet distribution that assigns documents to topics. For alpha asymmetric, we might risk of having topics more probable than others. For alpha smaller than 1, we are assuming the documents consist of a small number of topics, while for alpha greater than we assume the documents can be composed of more topics. In our case, it'd be better to have a symmetrical alpha, since we don't know if there's a topic much more likely than others.

For eta we apply the same logic but with topics and words. Eta smaller than 1 means we're assuming topics are not composed of many words, while Eta greater than 1 we assume more words constitute topics. In this case, it's better to let the model find the proper settings.

In [ ]:
# Fit an LDA model
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=6,
                     alpha='symmetric', eta='auto', passes=5, random_state=1)

auto_alpha = lda_model.alpha
auto_eta = lda_model.eta

print(f"Automatically determined alpha: {auto_alpha}")
print(f"Automatically determined eta: {auto_eta}")

Coherence is a measure of topics quality returned by the model and can be used for comparison

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, dictionary = id2word, texts=tokenized_docs, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print(f'LDA Coherence Score: {coherence_lda}')

In [ ]:
# Visualize the LDA model using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='tsne')
pyLDAvis.display(vis)

In [ ]:
lda_model[corpus]

# it returns a list of lists, where each inner list contains tuples where the topic ID and the topic probability are contained
# Each inner list is associated to a specific document

# output example
''' [
  DOC1: [(0, 0.05), (1, 0.09), ...]
  DOC2: [(0, 0.1), ...]
  .
  .
  .  
]
'''

In [ ]:
# this function returns the top words for each topic by highest frequency
lda_model.show_topic(0, topn=10)

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=df['Clean_Content']):
    # Init output
    sent_topics_df = pd.DataFrame(columns=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        # we want to sort the tuples in the list 'row' by descending order of the second element, which is the topic probability
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic, i.e. tuple with the highest topic probability
                wp = ldamodel.show_topic(topic_num, topn = 15)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df.loc[len(sent_topics_df)] = [int(topic_num), round(prop_topic, 4), topic_keywords]
            else:
                break
    # Add original text to the end of the output
    sent_topics_df = pd.concat([sent_topics_df, texts], axis=1)
    return sent_topics_df

df_topic_sents_keywords = format_topics_sentences()


In [ ]:
df_topic_sents_keywords

In [ ]:
df_topic_sents_keywords['Dominant_Topic'].unique()

In [ ]:
df_topic_distribution = df_topic_sents_keywords.groupby(['Dominant_Topic', 'Topic_Keywords']).size().sort_values(ascending=False).reset_index(name='count')

df_topic_distribution